# Segment and Convert  biosignals
Segment : cut the different acquisitions recorded in the same file (multi-run recordings).

Convert : change the file type (from `.acq` to `.tsv.gz`).


#### Dependencies

In [2]:
import pandas as pd
import numpy as np
import os

In [15]:
import neurokit2 as nk2
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
%pylab inline
sns.set_style('darkgrid')
pylab.rcParams['figure.figsize'] = (12.0, 6.0)  
plt.rcParams.update({'font.size': 14})

Populating the interactive namespace from numpy and matplotlib


#### import some code

In [4]:
%run ./code/conversion/test/convert_seg2hdf.py

In [5]:
%run code/utils/get_info.py -h

usage: get_info.py [-h] [-indir ROOT] [-sub SUB] [-ses SES] [-show SHOW]
                   [-save SAVE]

Required Argument::
  -indir ROOT, --input-directory ROOT
                        Specify root directory of dataset
  -sub SUB, --subject SUB
                        Specify alongside "-heur". Code of subject to process.

optional arguments:
  -h, --help            show this help message and exit
  -ses SES, --session SES
                        Specify alongside "-heur". Code of session to process.
  -show SHOW, --show-dict SHOW
                        Specify if you want to print dictionary
  -save SAVE, --save-dict SAVE
                        Specify if you want to save the dictionary


# Segmentation and Conversion



## `phys2bids` workflow
``phys2bids -in <input-file.acq> -chtrig <always-the-same-#> -ntp ### ### ### -tr 1.49 -outdir <root-bids-dir> -heur <path/to/heur.py>``

``-ntp`` : number of trigger timepoints listed by runs without brackets


In [6]:
ls = list_sub('/data/neuromod/DATA/cneuromod/movie10/sourcedata/physio', sub='sub-01', show=True)

list of files for session ses-001 ['sub-01_ses-bournesup01.acq']
list of files for session ses-002 ['sub-01_ses-bournesup02.acq']
list of files for session ses-003 ['sub-01_ses-wolfofwallstreet1.acq']
list of files for session ses-004 ['sub-01_ses-wolfofwallstreet2-3a.acq']
list of files for session ses-005 ['sub-01_ses-wolfofwallstreet2-3a.acq']
list of files for session ses-006 ['sub-01_ses-wolfofwallstreet.acq', 'sub-01_ses-life1.acq']
list of files for session ses-007 []
list of files for session ses-008 ['fast sampling rate data2019-05-03T15_41_57.acq']
list of files for session ses-009 ['fast sampling rate data2019-05-10T10_22_08.acq', 'fast sampling rate data2019-05-10T11_26_17.acq']
list of files for session ses-010 ['neuromod_video52019-05-29T15_39_51.acq']
list of files for session ses-011 ['neuromod_video52019-07-03T10_57_27.acq']
list of files for session ses-012 ['neuromod_video52019-08-19T15_18_25.acq']


++#### NOTE : have to keep sub in dataframe so i can group

In [9]:
metadata = pd.DataFrame()
tmp = pd.read_json(f'/home/francois.lespinasse/physiological-signals/tests/conversion/sub-06/sub-06_volumes_all-ses-runs.json')
for ses in tmp:
    dict_new = tmp[ses]
    metadata = metadata.append(dict_new, ignore_index=True)
    
metadata_whole = pd.DataFrame()
for sub in os.listdir('/home/francois.lespinasse/physiological-signals/tests/conversion/'):
    
    tmp = pd.read_json(f'/home/francois.lespinasse/physiological-signals/tests/conversion/{sub}/{sub}_volumes_all-ses-runs.json')
    
    for ses in tmp:
        dict_new = tmp[ses]
        metadata_whole = metadata_whole.append(dict_new, ignore_index=True)

### Number of volumes acquired per run for each sessions | sub-06

In [10]:
metadata

,expect_runs,in_file,processed_runs,run-01,run-02,run-03,run-04,run-05,run-06,run-07,run-08,run-09
0,4.0,sourcedata/physio/sub-06/ses-001/neuromod_vide...,4.0,410.0,410.0,410.0,410.0,NaN,NaN,NaN,NaN,NaN
1,5.0,sourcedata/physio/sub-06/ses-002/neuromod_vide...,5.0,410.0,411.0,409.0,408.0,410.0,NaN,NaN,NaN,NaN
2,5.0,sourcedata/physio/sub-06/ses-003/neuromod_vide...,5.0,410.0,410.0,410.0,410.0,374.0,NaN,NaN,NaN,NaN
3,1.0,sourcedata/physio/sub-06/ses-004/neuromod_vide...,1.0,410.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7.0,sourcedata/physio/sub-06/ses-005/neuromod_vide...,7.0,409.0,391.0,403.0,405.0,405.0,405.0,405.0,NaN,NaN
5,9.0,sourcedata/physio/sub-06/ses-006/neuromod_vide...,9.0,405.0,405.0,405.0,405.0,380.0,404.0,409.0,411.0,409.0
6,3.0,sourcedata/physio/sub-06/ses-007/neuromod_vide...,3.0,410.0,408.0,409.0,NaN,NaN,NaN,NaN,NaN,NaN
7,3.0,sourcedata/physio/sub-06/ses-008/neuromod_vide...,3.0,410.0,410.0,373.0,NaN,NaN,NaN,NaN,NaN,NaN
8,7.0,sourcedata/physio/sub-06/ses-009/neuromod_vide...,7.0,407.0,407.0,407.0,407.0,407.0,407.0,407.0,NaN,NaN
9,8.0,sourcedata/physio/sub-06/ses-010/neuromod_vide...,8.0,407.0,407.0,407.0,407.0,407.0,407.0,407.0,504.0,NaN


##### See where there are missing runs for all phys data acquired

In [38]:
mismatch = list(np.where(np.array(metadata_whole['expect_runs']-metadata_whole['processed_runs'])>0)[0]
)

In [46]:
print(metadata_whole['in_file'][mismatch[0:-1]])

22    sourcedata/physio/sub-03/ses-012/neuromod_vide...
23    sourcedata/physio/sub-03/ses-013/neuromod_vide...
49    sourcedata/physio/sub-02/ses-004/sub-02_ses-bo...
55    sourcedata/physio/sub-02/ses-010/neuromod_vide...
Name: in_file, dtype: object


### homebrewed method

In [9]:
help(batch_parse)

Help on function batch_parse in module __main__:

batch_parse(root, subject, ses=None, save_path=None)
    Automated signal parsing for biopac recordings following BIDS format.
    
    Make sure the trigger channel is named "TTL" because it is hard-coded
    Parameters:
    ------------
    root : path
        main directory containing the biopac data (e.g. /home/user/dataset)
    subject : string
        name of path for a specific subject (e.g.'sub-03')
    ses : string
        name of acquisition session. Optional workflow for specific experiment
        default is None
    save_path: path
        root directory of

